In [16]:
from sklearn.decomposition import randomized_svd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [17]:
#user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
#place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
#product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
#video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
#user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
#user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템
#index_user_place = pd.read_csv('../Data/whyout_data/index_user_place.csv', index_col=0)
#index_user_product = pd.read_csv('../Data/whyout_data/index_user_product.csv', index_col=0)
index_user_video = pd.read_csv('../Data/whyout_data/index_user_video.csv', index_col=0)

In [18]:
# 데이터프레임을 numpy matrix로 만듦
user_product_matrix = user_product.values

# 사용자의 평균 평점
user_ratings_mean = np.mean(user_product_matrix, axis=1)

# 사용자-상품에 대해 사용자 평균 평점을 뺀 것
matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1,1)

In [19]:
pd.DataFrame(matrix_user_mean, columns = user_product.columns).head()

,1,2,3,4,5,6,7,8,9,10,...,5825,5826,5827,5828,5829,5830,5831,5832,5833,5834
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,...,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514,-0.000514


In [20]:
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# U, sigma, Vt = svds(matrix, k = 20)
U, sigma, Vt = randomized_svd(user_product_matrix, n_components=10)

# 특이값 벡터를 대각 행렬 형태로 변환해줌
sigma = np.diag(sigma)

In [21]:
# U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.matmul(np.matmul(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [22]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_product.columns)

In [23]:
product

,idx,카테고리1,카테고리2,색상,기능성(합집합),휴대성,친환경,디자인,클릭 수,좋아요 수,북마크 수
0,2,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.000000,0.017857
1,3,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.000000
2,4,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.017857
3,5,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.000000,0.000000
4,6,"[0,0,0,0,1,0,0,0,0,0,0]","[1,0,0]","[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]","[[1,0,0,0,0,0]]","[[0,0,1,0]]","[[1,0,0],[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[...",NaN,0.015873,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5829,6369,"[0,0,1,0,0,0,0,0,0,0,0]","[0,0,1]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0],[0,1,0,0,0,0]]","[[0,1,0,0]]","[[1,0,0],[0,1,0]]",NaN,NaN,0.000000,0.000000
5830,6370,"[0,0,1,0,0,0,0,0,0,0,0]","[0,0,1]","[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[[1,0,0,0,0,0]]","[[1,0,0,0]]","[[0,1,0]]",NaN,NaN,0.000000,0.000000
5831,6371,"[0,1,0,0,0,0,0,0,0,0,0]","[0,1,0]",NaN,"[[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0]]",NaN,"[[1,0,0]]",NaN,NaN,0.000000,0.000000
5832,6372,"[0,1,0,0,0,0,0,0,0,0,0]","[0,1,0]",NaN,"[[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0]]",NaN,"[[1,0,0]]",NaN,NaN,0.000000,0.000000


In [24]:
user_product

,1,2,3,4,5,6,7,8,9,10,...,5825,5826,5827,5828,5829,5830,5831,5832,5833,5834
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_svd_preds

,1,2,3,4,5,6,7,8,9,10,...,5825,5826,5827,5828,5829,5830,5831,5832,5833,5834
0,-1.392857e-18,-3.956735e-16,8.538308e-19,3.976289e-17,-5.301938e-17,-3.287865e-17,-1.253981e-17,6.213822e-31,6.934750e-31,-8.700113e-31,...,-3.364088e-19,0.000000,9.169166e-21,0.000000,-1.557366e-21,-1.557366e-21,0.000000,1.679177e-20,5.289897e-24,-6.931238e-17
1,7.114442e-19,1.183703e-16,1.645113e-18,8.497420e-16,4.773068e-18,6.185721e-17,2.682562e-16,1.000788e-30,3.396489e-31,-2.726169e-30,...,8.233579e-19,0.000000,-1.713045e-21,0.000000,9.652007e-22,9.652007e-22,0.000000,8.985320e-21,2.504760e-24,2.907625e-18
2,1.224439e-18,-1.788500e-16,6.326450e-18,3.294519e-15,-3.648575e-17,1.914259e-16,1.004320e-15,4.214912e-30,1.736578e-30,-1.129683e-29,...,2.897696e-18,0.000000,3.646778e-21,0.000000,2.659008e-21,2.659008e-21,0.000000,4.627772e-20,1.434668e-23,4.855247e-17
3,-1.367871e-18,-2.411897e-16,-3.034134e-18,6.014381e-16,2.154189e-17,3.140503e-17,1.736124e-16,-2.131363e-31,-5.418638e-31,3.001679e-31,...,4.649905e-19,0.000000,-5.769040e-21,0.000000,-7.129049e-22,-7.129049e-22,0.000000,-5.651920e-21,-2.397340e-24,2.882294e-18
4,5.142272e-04,5.142525e-04,5.142280e-04,5.142437e-04,5.142267e-04,5.142284e-04,5.142326e-04,5.142269e-04,5.142269e-04,5.142269e-04,...,5.142272e-04,0.000514,5.142269e-04,0.000514,5.142269e-04,5.142269e-04,0.000514,5.142270e-04,5.142269e-04,5.142353e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31171,5.142291e-03,5.148084e-03,5.142314e-03,5.142998e-03,5.142212e-03,5.142657e-03,5.142816e-03,5.142269e-03,5.142269e-03,5.142269e-03,...,5.142282e-03,0.005142,5.142269e-03,0.005142,5.142269e-03,5.142269e-03,0.005142,5.142270e-03,5.142269e-03,5.142876e-03
31172,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
31173,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
31174,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00


In [26]:
df_svd_preds.head(7)

,1,2,3,4,5,6,7,8,9,10,...,5825,5826,5827,5828,5829,5830,5831,5832,5833,5834
0,-1.392857e-18,-3.956735e-16,8.538308e-19,3.976289e-17,-5.301938e-17,-3.287865e-17,-1.253981e-17,6.213822e-31,6.934750e-31,-8.700113e-31,...,-3.364088e-19,0.000000,9.169166e-21,0.000000,-1.557366e-21,-1.557366e-21,0.000000,1.679177e-20,5.289897e-24,-6.931238e-17
1,7.114442e-19,1.183703e-16,1.645113e-18,8.497420e-16,4.773068e-18,6.185721e-17,2.682562e-16,1.000788e-30,3.396489e-31,-2.726169e-30,...,8.233579e-19,0.000000,-1.713045e-21,0.000000,9.652007e-22,9.652007e-22,0.000000,8.985320e-21,2.504760e-24,2.907625e-18
2,1.224439e-18,-1.788500e-16,6.326450e-18,3.294519e-15,-3.648575e-17,1.914259e-16,1.004320e-15,4.214912e-30,1.736578e-30,-1.129683e-29,...,2.897696e-18,0.000000,3.646778e-21,0.000000,2.659008e-21,2.659008e-21,0.000000,4.627772e-20,1.434668e-23,4.855247e-17
3,-1.367871e-18,-2.411897e-16,-3.034134e-18,6.014381e-16,2.154189e-17,3.140503e-17,1.736124e-16,-2.131363e-31,-5.418638e-31,3.001679e-31,...,4.649905e-19,0.000000,-5.769040e-21,0.000000,-7.129049e-22,-7.129049e-22,0.000000,-5.651920e-21,-2.397340e-24,2.882294e-18
4,5.142272e-04,5.142525e-04,5.142280e-04,5.142437e-04,5.142267e-04,5.142284e-04,5.142326e-04,5.142269e-04,5.142269e-04,5.142269e-04,...,5.142272e-04,0.000514,5.142269e-04,0.000514,5.142269e-04,5.142269e-04,0.000514,5.142270e-04,5.142269e-04,5.142353e-04
5,5.866901e-19,8.066841e-17,2.572675e-18,1.145388e-16,-5.559306e-18,6.316287e-18,4.005554e-17,3.260619e-30,2.458939e-30,-8.625677e-30,...,-2.638961e-19,0.000000,4.300270e-21,0.000000,7.272576e-22,7.272576e-22,0.000000,-3.771929e-21,-1.281392e-24,5.884406e-17
6,-6.190349e-19,-2.336401e-17,1.529873e-18,-3.948267e-16,-2.728030e-17,-4.227771e-17,-1.305432e-16,8.908670e-31,1.284540e-30,-1.400768e-30,...,1.863987e-19,0.000000,2.956088e-21,0.000000,-1.219353e-21,-1.219353e-21,0.000000,2.294039e-20,9.325550e-24,-9.829588e-17


In [27]:
def recommend_movies(df_svd_preds, user_id, df, ratings_df, num_recommendations):
    
    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_data = ratings_df.loc[user_id]  # Series 형태로 가져오기
    #print(user_data)

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = user_data[user_data > 0].index.tolist()
    print(user_history_indices)

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    # 현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = df.loc[~df.index.isin(user_data[user_history_indices])]
    print(len(recommendations))
    print(len(df))

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    recommendations = recommendations.merge(predictions_df, left_index=True, right_on='index')

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_movies(df_svd_preds, 5, product, user_product, 5)

['519']
5833
5834


In [28]:
predictions

,idx,카테고리1,카테고리2,색상,기능성(합집합),휴대성,친환경,디자인,클릭 수,좋아요 수,북마크 수,index,Predictions
0,3881,"[0,0,0,0,0,0,0,0,1,0,0]","[0,1,0]","[[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]]","[[0,0,1,0,0,0]]","[[0,0,1,0]]","[[0,1,0]]","[[0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0],[...",NaN,0.000000,0.017857,3580,0.000525
1,1376,"[0,0,0,0,0,0,0,0,0,0,1]","[0,0,1]","[[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]","[[1,0,0,0,0,0],[0,0,1,0,0,0]]","[[0,0,1,0]]","[[0,1,0]]","[[0,0,0,0,0,0,0,1,0,0]]",NaN,0.000000,0.035714,1202,0.000520
2,3521,"[1,0,0,0,0,0,0,0,0,0,0]","[1,0,0]",NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,3297,0.000518
3,3832,"[0,0,0,0,0,1,0,0,0,0,0]","[1,0,0]",NaN,"[[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0]]","[[0,1,0,0]]","[[0,1,0]]","[[0,1,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0],[...",NaN,0.000000,0.000000,3531,0.000517
4,1374,"[0,0,0,0,0,0,0,0,0,0,1]","[0,0,1]","[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]","[[0,0,1,0,0,0]]","[[0,0,1,0]]","[[0,1,0]]","[[1,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0]]",NaN,0.015873,0.107143,1200,0.000517
